# Proyecto 7 Creacion de un Modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

## Introducción 
Utilizaremos el conjunto de datos `/datasets/users_behavior.csv` para poder entrenar un modelo que recomiende a los clientes el mejor plan para ellos con base a su historial. Para lograr esto llevaremos a cabo las siguientes etapas en el orden que se describen.

### Etapas
1. Importaremos las librerias y abriremos y examinaremos el archivo de datos.
    * 1.1 Importar Librerias
    * 1.2 Abrir archivo de datos y examinarlo.
2. Segmentaremos los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
3. Investigaremos la calidad de diferentes modelos cambiando los hiperparámetros. Describiremos brevemente los hallazgos del estudio.
    * 3.1 Modelo Árbol de decisión de regresión
    * 3.2 Modelo Bosque aleatorio de regresión
    * 3.3 Modelo Regresión lineal
4. Comprobaremos la calidad del modelo usando el conjunto de prueba.
5. Prueba de cordura al modelo
6. Conclusion


# Etapa 1 Importacion de librerias y examinacion del archivo de datos.

## 1.1  Importar Librerias

In [1]:
# Importaremos la libreria pandas y los modelos necesarios para poder crear nuestros modelos.
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error

## 1.2 Abrir archivo de datos y examinarlo

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()
display(df.head(5))
print('\nNumero de valores duplicados:',df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0



Numero de valores duplicados: 0


**OBSERVACIONES**
Nuestro conjunto de datos esta listo para poder llevar a cabo la creación de modelos ya que no tenemos valores faltantes ni duplicados. La columna `calls` contiene el numero de llamadas, la columna `minutes` contiene la duración de las llamadas en minutos, la columna `messages` contiene el numero de mensajes de texto, la columna `mb_used` contiene la cantidad de megas usados y la columna `is_ultra` es una columna binario de zeros y unos donde el 0 representa que tiene el plan `Smart` y el 1 representa que tienen el plan `Ultra`.

# Etapa 2 Segmentacion de los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [4]:
# Creamos features y target

features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

# Primero dividimos nuestro conjunto de datos en 60% para el entrenamiento, 20 para el conjunto de validacion
# 20 para el conjunto de prueba.

features_train, features_valid, target_train, target_valid = train_test_split(features, 
target, test_size= .4 , random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
target_valid, test_size= .50 , random_state=12345)

para dividir nuestro conjunto de datos utilizamos `train_test_split` El cual nos permite dividir nuestro conjunto de datos en 2 porciones y esas porciones se basaran en el porcentaje que nosotros indiquemos en `test_size`. Usualmente, el tamaño del conjunto de validación y del de prueba son iguales. Lo que nos da una proporción de datos fuente de 3:1:1 y esto seria una proporcion de 60% de los datos para el conjunto de entrenamiento y `20%` para el conjunto de validacion y `20%` para el conjunto de prueba. Para lograr esto dividimos el conjunto de datos de tal forma que el conjunto de entrenamiento se quede con el 60 porciento y el conjunto de validacion con un 40 porciento para despues dividirlo en la mitad y que el conjunto de prueba tenga un 20 porciento al igual que el conjunto de validacion.

# Etapa 3 Investigaremos la calidad de diferentes modelos cambiando los hiperparámetros. Describiremos brevemente los hallazgos del estudio.

## 3.1 Modelo Árbol de decisión de regresión

In [5]:
# Identificaremos los mejores hiperparamtros para nuestro modelo de Arbol de decision de regresion.

best_model = None
best_result = 0 
best_depth = 0
for depth in range(1, 6): # seleccionaremos el rango del hiperparámetro
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
    model.fit(features_train, target_train) # entrenaremos el modelo con el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid) 
    result =  accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print(f"Umbral de exactitud del mejor modelo con el conjunto de validación (max_depth = {best_depth}): {best_result}")

Umbral de exactitud del mejor modelo con el conjunto de validación (max_depth = 3): 0.7853810264385692


Para poder decidir cual seria la mejor profundidad de nuestro modelo de Arbol utilizamos for para que tomara un rango entre 1 y 5 ya que si aumentamos nuestro rango no vemos una mejora significativa en el resultado y solo haria el proceso mas lento por lo cual decidimos optar por ese rango de profundidad. Al hacer esto recibimos como respuesta que la mejor profundidad es 3 y el mejor resultado que nos da es un 78.5% de exactitud lo cual no esta nada mal ya que supera nuestro umbral de exactitud establecido en un 75%, seguiremos probrando los demas modelos para ver si existe uno que nos de una mejor exactitud.

##  3.2 Modelo Bosque aleatorio de regresión

In [6]:
# Crearemos nuestro Modelo Bosque aleatorio de regresion

best_model_rf = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # entrenaremos el modelo con el conjunto de entrenamiento
        predictions_valid = model.predict(features_valid) # obténdremos las predicciones del modelo con el conjunto de validación
        result = accuracy_score(target_valid, predictions_valid) 
        if result > best_result:
            best_model_rf = model
            best_result = result
            best_est = est
            best_depth = depth

print("Umbral de exactitud del mejor modelo con el conjunto de validación", best_result, "n_estimators:", best_est, "best_depth:", depth)

Umbral de exactitud del mejor modelo con el conjunto de validación 0.8087091757387247 n_estimators: 40 best_depth: 9


Wow este modelo tiene una exactitud del 80.8% es superior al del modelo anterior por lo que podemos descartar al modelo anterior. En este caso estamos probando un modelo de bosque y utilizamos for para poner un rango de de 10 a 50, en intervalos de 10 y esto lo usamos para poder decidir cual seria la mejor cantidad de arboles que nosotros seleccionamos con n_estimators por lo cual n debe de estar en ese rango que acabamos de definir y utilizamos ese rango debido a que si aumentamos el rango no vemos una diferencia significativa. Tambien utilizamos for para poder descubrir cual es el mejor rango de profundidad entre 1 y 10 y de la misma forma que explicamos si utilizamos un rango mas grande no vemos una diferencia significativa y es por eso que dejamos ese rango. Al recibir nuestra respuesta tenemos un umbral de exactitud del `80.8%`, obtenemos que la mejor cantidad de arboles son 40 con una profundidad de 9.

## 3.3 Modelo Regresión logistica

In [7]:
#Crearemos nuestro Modelo de Regresion lineal

model = LogisticRegression()
model.fit(features_train, target_train) # entrenaremos el modelo con el conjunto de entrenamiento
predictions_valid = model.predict(features_valid) # obténdremos las predicciones del modelo con el conjunto de validación

result = accuracy_score(target_valid, predictions_valid)
print( "Umbral de exactitud del modelo de regresión logistica en el conjunto de validación:", result)

Umbral de exactitud del modelo de regresión logistica en el conjunto de validación: 0.7107309486780715


Nuestro modelo de Regresion Logistica obtuve el porcentaje mas bajo de exactitud con un `71%` por lo cual tambien lo descartaremos y optaremos por usar el modelo de bosque.

# Etapa 4 Comprobaremos la calidad del mejor modelo usando el conjunto de prueba.

In [8]:
# utilizaremos el modelo bosque aleatorio de regresion con el conjunto de prueba

predictions_test = best_model_rf.predict(features_test) 
result =  accuracy_score(target_test, predictions_test)
print(result)


0.7962674961119751


Al utilizar el conjunto de prueba con nuestro mejor modelo que habiamos guardado en la variable `best_model_rf` que representa el modelo de bosque aleatorio de regresion obtenemos un `79.6%` de exactitud el cual supera el `75%` de umbral de exatitud que se nos requeria. EL porcentaje de exactitud bajo un poco con el conjunto de prueba pero es normal que eso pase y aun asi sigue superando el umbral de exatitud de los modelos de `arbol` y `linear`.

# Etapa 5 Comprobaremos la cordura del modelo.

In [9]:
# Crearemos un modelo DummyClassifier para poder hacer la prueba de cordura del modelo.
dummy_model = DummyClassifier(strategy="most_frequent")


dummy_model.fit(features_train, target_train) # entrenaremos el modelo con el conjunto de entrenamiento.

predictions_dummy = dummy_model.predict(features_test) # obtendremos las predicciones con el conjunto de prueba.

predictions_best_model = best_model_rf.predict(features_test) # obtener las predicciones del mejor modelo con el mismo conjunto.

# Comparar la exacitud de los modelos.
accuracy_dummy = accuracy_score(target_test, predictions_dummy)
accuracy_best_model = accuracy_score(target_test, predictions_best_model)

# Imprimir las exactitudes
print("Exactitud del Dummy Classifier:", accuracy_dummy)
print("Exactitud del Mejor Modelo:", accuracy_best_model)

Exactitud del Dummy Classifier: 0.6842923794712286
Exactitud del Mejor Modelo: 0.7962674961119751


Nuestro mejor modelo supera de una manera significativa el modelo DummyClassifier, lo cual nos dice que nuestro modelo si funciona y es una mejor opcion que simplemente adoptar una estrategia simple. Nuestro modelo tiene un buen porcentaje de exactitud y es capaz de aprender de los patrones que se hallan en los datos y tiene la capacidad de hacer predicciones.

# Etapa 6 Conclusion.

Algo que aprendi en este proyecto es que siempre va a ser muy bueno dividir la carga de trabajo del proyecto en partes pequeñas. En este caso pudimos enfocarnos solamente en crear los modelos y compararlos, fue mucho mas eficaz que tener todo el trabajo de limpiar los datos y todo eso y quizas nuestra mente no se enfocaria de la mejor forma, pero al dividir el trabajo en partes todo es mas facil. De este analisis me gustaria destacar 3 puntos principales.

1. La exactitud de nuestro modelo es del 80.8% con nuestro conjunto de entrenamiento y 79.6% con nuestro conjunto de prueba con lo cual ambos superan la meta establecida de un 75% de exactitud.


2. El mejor modelo es un Bosque aleatorio de regresion con 40 arboles y 9 de profundidad.


3. Nuestro modelo supera de forma significativa al modelo Dummy por lo cual nuestro modelo si funciona y es util ya que tiene mejor desempeño que optar por una estrategia simple.